In [69]:
import os
import re
import struct


def parse_part_mapping(file_path):
    part_mapping = {}
    current_category = None

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip()
            if not line:
                continue  # Skip empty lines

            # Detect category header like 'Head (0):'
            if line.endswith('):'):
                category_name, category_id = line[:-2].rsplit('(', 1)
                current_category = category_name.strip()
                part_mapping[current_category] = {}
                continue

            if current_category is None:
                continue  # Skip any lines before first category

            # Split the line into part_id and part_name
            if ' ' in line:
                part_id, part_name = line.split(' ', 1)
                part_mapping[current_category][part_id.strip()
                                               ] = part_name.strip()

    return part_mapping


part_mapping = parse_part_mapping("ACFA_PS3_US_PARTID_TO_PARTNAME.txt")
print(part_mapping.keys())

BLOCK_SIZE = 24280
NAME_SIZE = 96  # 48 wchar_t = 96 bytes in UTF-16


def load_file(path):
    with open(path, "rb") as f:
        return f.read()


def save_file(path, data):
    with open(path, "wb") as f:
        f.write(data)

def linear_utf16_clean_name_reader(data, start_offset, max_bytes=96):
    raw_field = data[start_offset:start_offset + max_bytes]
    try:
        decoded = raw_field.decode('utf-16-le', errors='ignore').strip('\x00')
        match = re.match(r'^[A-Za-z0-9 ]+', decoded)
        if match:
            return match.group(0).strip()
        return "<Invalid UTF-16 Encoding>"
    except UnicodeDecodeError:
        return "<Invalid UTF-16 Encoding>"


def read_timestamp(data, offset):
    timestamp_bytes = data[offset:offset + 8]
    return struct.unpack(">Q", timestamp_bytes)[0]


def extract_active_schematic_blocks(file_path):
    """
    Extracts all schematic blocks from the given file.
    Returns a list of blocks.
    """
    data = load_file(file_path)
    schematic_count = data[5]
    blocks = []
    first_marker_offset = 0x148

    for slot_index in range(schematic_count):
        block_start = first_marker_offset + (slot_index * BLOCK_SIZE)
        block = data[block_start:block_start + BLOCK_SIZE]
        blocks.append(block)
    return blocks


def display_schematic_info(block):
    """
    Displays the schematic information from a block.
    Returns a dictionary with the schematic information.
    """
    schematic_name = linear_utf16_clean_name_reader(block, 1, NAME_SIZE)
    designer_name = linear_utf16_clean_name_reader(
        block, 1 + NAME_SIZE, NAME_SIZE)
    timestamp = read_timestamp(block, 192)

    protect_category_byte = block[200]
    protect = (protect_category_byte & 0b10000000) >> 7
    category = (protect_category_byte & 0b01111111) + 1

    parts = extract_parts(block, part_mapping)
    tuning = extract_tuning(block)

    schematic_info = {
        "name": schematic_name,
        "designer": designer_name,
        "category": category,
        "timestamp": timestamp,
        "parts": parts,
        "tuning": tuning
    }

    # Optional print for debugging
    # print(f"  Name: {schematic_info['name']}")
    # print(f"  Designer: {schematic_info['designer']}")
    # print(f"  Protect: {schematic_info['protect']}")
    # print(f"  User Slot: {schematic_info['category']}")
    # print(f"  Raw Byte: 0x{schematic_info['raw_byte']:02x}")
    # print(f"  Timestamp: {schematic_info['timestamp']}")
    # print("  Parts:", schematic_info['parts'])
    # print("  Tuning:", schematic_info['tuning'])
    # print()

    return schematic_info


def extract_parts(block, part_name_lookup):
    LOCAL_PARTS_OFFSET = 0xD8  # 0x220 - 0x148
    PART_ENTRY_SIZE = 2

    # Define lookup keys and display labels separately
    lookup_keys = [
        'Head', 'Core', 'Arms', 'Legs', 'FCS', 'Generator', 'Main Booster',
        'Back Booster', 'Side Booster', 'Overed Booster',
        'Arm Unit', 'Arm Unit', 'Back Unit', 'Back Unit', 'Shoulder Unit'
    ]

    display_labels = [
        'Head', 'Core', 'Arms', 'Legs', 'FCS', 'Generator', 'Main Booster',
        'Back Booster', 'Side Booster', 'Overed Booster',
        'Right Arm Unit', 'Left Arm Unit', 'Right Back Unit',
        'Left Back Unit', 'Shoulder Unit'
    ]

    parts_info = []
    for i, (lookup_key, display_label) in enumerate(zip(lookup_keys, display_labels)):
        offset = LOCAL_PARTS_OFFSET + i * PART_ENTRY_SIZE
        part_id_bytes = block[offset:offset + PART_ENTRY_SIZE]

        if len(part_id_bytes) != 2:
            part_id_str = "<Invalid>"
            part_name = "<Invalid>"
        else:
            part_id_num = int.from_bytes(part_id_bytes, byteorder='big')
            part_id_str = f"{part_id_num:04d}"
            part_name = part_name_lookup.get(lookup_key, {}).get(
                part_id_str, f"Unknown ID {part_id_str}")

        parts_info.append({
            "category": display_label,
            "part_id": part_id_str,
            "part_name": part_name
        })

    return parts_info


def extract_tuning(block):
    LOCAL_TUNING_OFFSET = 0x126  # 0x26E - 0x148
    TUNING_SIZE = 32  # 0x20 bytes

    tuning_labels = [
        'en_output',
        'en_capacity',
        'kp_output',
        'load',
        'en_weapon_skill',
        'maneuverability',
        'firing_stability',
        'aim_precision',
        'lock_speed',
        'missile_lock_speed',
        'radar_refresh_rate',
        'ecm_resistance',
        'rectification_head',
        'rectification_core',
        'rectification_arm',
        'rectification_leg',
        'horizontal_thrust_main',
        'vertical_thrust',
        'horizontal_thrust_side',
        'horizontal_thrust_back',
        'quick_boost_main',
        'quick_boost_side',
        'quick_boost_back',
        'quick_boost_overed',
        'turning_ability',
        'stability_head',
        'stability_core',
        'stability_legs',
    ]

    tuning_values = {}
    for i, label in enumerate(tuning_labels):
        value = block[LOCAL_TUNING_OFFSET + i]
        tuning_values[label] = value  # Value should be in range 0-50

    return tuning_values



def save_schematic_block_as_ac4a(hex_block: bytes):
    """
    Saves a single schematic block to  'output/{schematic_name}_{designer_name}.ac4a' file.
    """
    sch_data = display_schematic_info(hex_block)
    schematic_name = sch_data['name']
    designer_name = sch_data['designer']
    output_path = f"output/{schematic_name}_{designer_name}.ac4a"

    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, "wb") as f:
        f.write(hex_block)


def load_schematic_block_from_ac4a(file_path: str) -> bytes:
    """
    Loads a schematic block from a .ac4a file.
    Returns the raw bytes representing the schematic block.
    """
    with open(file_path, "rb") as f:
        return f.read()
    

def insert_schematic(ac4a_path, desdoc_path):
    BLOCK_SIZE = 24280
    SCHEMATIC_COUNT_OFFSET = 5
    FIRST_MARKER_OFFSET = 0x148

    # Load files
    ac4a_data = load_file(ac4a_path)
    desdoc_data = bytearray(load_file(desdoc_path))

    # Extract current schematic count
    current_count = desdoc_data[SCHEMATIC_COUNT_OFFSET]
    print(f"Current active schematic count: {current_count}")

    # Calculate insertion offset
    insertion_offset = FIRST_MARKER_OFFSET + (current_count * BLOCK_SIZE)

    # Validate insertion space
    if insertion_offset + BLOCK_SIZE > len(desdoc_data):
        print("Error: Not enough space to insert schematic.")
        return

    # Insert schematic data
    desdoc_data[insertion_offset:insertion_offset +
                BLOCK_SIZE] = ac4a_data[:BLOCK_SIZE]

    # Update schematic count
    desdoc_data[SCHEMATIC_COUNT_OFFSET] += 1
    print(
        f"Inserted at slot {current_count + 1} (offset {hex(insertion_offset)}). New count: {desdoc_data[SCHEMATIC_COUNT_OFFSET]}")

    # Save modified DESDOC.DAT
    save_file(desdoc_path, desdoc_data)

dict_keys(['Head', 'Core', 'Arms', 'Legs', 'FCS', 'Generator', 'Main Booster', 'Back Booster', 'Side Booster', 'Overed Booster', 'Arm Unit', 'Back Unit', 'Shoulder Unit', 'Stabilizer Head Top', 'Stabilizer Head Side', 'Stabilizer Core Upper', 'Stabilizer Core Lower', 'Stabilizer Arms', 'Stabilizer Legs Back', 'Stabilizer Legs Upper', 'Stabilizer Legs Middle', 'Stabilizer Legs Lower'])


In [70]:
loaded_block = load_schematic_block_from_ac4a(
    "output/Sbeu Tarakan_Vlabus.ac4a")

# Verify loaded block content by displaying its information
display_schematic_info(loaded_block)

{'name': 'Sbeu Tarakan',
 'designer': 'Vlabus',
 'category': 2,
 'timestamp': 63883041354520147,
 'parts': [{'category': 'Head', 'part_id': '3010', 'part_name': '047AN02'},
  {'category': 'Core', 'part_id': '4010', 'part_name': 'C01-TELLUS'},
  {'category': 'Arms', 'part_id': '0051', 'part_name': 'ARGYROS/A'},
  {'category': 'Legs', 'part_id': '0230', 'part_name': 'GAEN01-SL-L'},
  {'category': 'FCS', 'part_id': '3021', 'part_name': '061AN05'},
  {'category': 'Generator', 'part_id': '2010', 'part_name': '03-AALIYAH/G'},
  {'category': 'Main Booster', 'part_id': '2010', 'part_name': 'S04-VIRTUE'},
  {'category': 'Back Booster', 'part_id': '4021', 'part_name': 'BB11-LATONA'},
  {'category': 'Side Booster',
   'part_id': '4010',
   'part_name': 'SB128-SCHEDAR'},
  {'category': 'Overed Booster', 'part_id': '1010', 'part_name': 'KB-JUDITH'},
  {'category': 'Right Arm Unit',
   'part_id': '0151',
   'part_name': 'ALLEGHENY01'},
  {'category': 'Left Arm Unit', 'part_id': '4091', 'part_name': 

In [76]:
file_path = "sch_data/tab_sch_data/DESDOC.DAT"
blocks = extract_active_schematic_blocks(file_path)
print(f"Detected {len(blocks)} active schematic(s).\n")

for idx, block in enumerate(blocks, 1):
    print(f"Slot {idx}:")
    print(display_schematic_info(block))

Detected 10 active schematic(s).

Slot 1:
{'name': 'U1TEST', 'designer': 'Unknown', 'category': 1, 'timestamp': 63882922127095465, 'parts': [{'category': 'Head', 'part_id': '1041', 'part_name': 'HD-LANCEL'}, {'category': 'Core', 'part_id': '1021', 'part_name': 'CR-LANCEL'}, {'category': 'Arms', 'part_id': '1031', 'part_name': 'AM-LANCEL'}, {'category': 'Legs', 'part_id': '1031', 'part_name': 'LG-LANCEL'}, {'category': 'FCS', 'part_id': '1010', 'part_name': 'FS-HOGIRE'}, {'category': 'Generator', 'part_id': '1010', 'part_name': 'GN-HOGIRE'}, {'category': 'Main Booster', 'part_id': '1020', 'part_name': 'CB-HOGIRE'}, {'category': 'Back Booster', 'part_id': '1010', 'part_name': 'LB-HOGIRE'}, {'category': 'Side Booster', 'part_id': '1030', 'part_name': 'AB-HOGIRE'}, {'category': 'Overed Booster', 'part_id': '1031', 'part_name': 'KRB-PALLAS'}, {'category': 'Right Arm Unit', 'part_id': '5071', 'part_name': 'LABIATA'}, {'category': 'Left Arm Unit', 'part_id': '1010', 'part_name': 'RF-R100'}, {

In [58]:
save_schematic_block_as_ac4a(blocks[6])

In [75]:
insert_schematic("output/Sbeu Tarakan_Vlabus.ac4a",
                 "sch_data/tab_sch_data/DESDOC.DAT")

Current active schematic count: 9
Inserted at slot 10 (offset 0x356e0). New count: 10
